<a href="https://colab.research.google.com/github/NITHIN-KESHAV/DATA-228-Group-Project/blob/main/Fine_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch accelerate weaviate-client


In [ ]:
import torch
if not torch.cuda.is_available():
    raise EnvironmentError("GPU is not available. Please ensure you are using a GPU runtime.")
else:
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")


Using GPU: NVIDIA A100-SXM4-40GB


In [ ]:
import weaviate

# Weaviate URL
weaviate_url = "http://50.18.99.196:8080"

# Initialize client
client = weaviate.Client(weaviate_url)
print("Connected to Weaviate!")


<ipython-input-27-a8d44a56cd12>:7: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(weaviate_url)


Connected to Weaviate!


In [ ]:
def load_reviews_for_summarization(limit=10000):
    """
    Load reviews for summarization from Weaviate with a limit.
    """
    query = client.query.get(
        "Review", ["reviewText_cleaned", "summary_cleaned"]
    ).with_limit(limit).do()

    # Debug: Print query response
    print(f"Query Response: {query}")

    reviews = query.get("data", {}).get("Get", {}).get("Review", [])
    if not reviews:
        raise ValueError("No data retrieved from Weaviate. Check your schema and connection.")

    # Prepare dataset
    dataset = [
        {"input_text": review["reviewText_cleaned"], "target_text": review["summary_cleaned"]}
        for review in reviews
        if review.get("reviewText_cleaned") and review.get("summary_cleaned")
    ]
    return dataset

# Load data
reviews = load_reviews_for_summarization(limit=10000)
print(f"Loaded {len(reviews)} reviews for training.")


Query Response: {'data': {'Get': {'Review': [{'reviewText_cleaned': 'soft tshirt material very comfortable size large was on the bigger side', 'summary_cleaned': 'very comfortable size large was on the bigger side'}, {'reviewText_cleaned': 'these drops were more like samples than a fullfledged set of sweeteners that could be used for any length of time taste was fine but just not very much in the bottles', 'summary_cleaned': 'product taste find just need more in the bottle'}, {'reviewText_cleaned': 'received on time the product is good quality and easy to put together it looks great and holds more than i thought it would', 'summary_cleaned': 'the product is good quality and easy to put together'}, {'reviewText_cleaned': 'ok for what i wanted', 'summary_cleaned': 'ok for what i wanted'}, {'reviewText_cleaned': 'beautiful', 'summary_cleaned': 'five stars'}, {'reviewText_cleaned': 'my husband loves these tshirts', 'summary_cleaned': 'five stars'}, {'reviewText_cleaned': 'i am very impress

In [ ]:
from datasets import Dataset

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(reviews)

# Split into train and validation sets
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(f"Dataset loaded and split into training ({len(train_dataset)}) and validation ({len(eval_dataset)}) sets.")


Dataset loaded and split into training (7995) and validation (1999) sets.


In [ ]:
from transformers import T5Tokenizer

# Load Tokenizer
model_name = "t5-small"  # Replace with your desired model, e.g., "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
print("Tokenizer loaded.")


Tokenizer loaded.


In [ ]:
def preprocess_function(examples):
    inputs = examples["input_text"]
    targets = examples["target_text"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize target text
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize datasets
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True)

print("Datasets tokenized.")


Map:   0%|          | 0/7995 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Datasets tokenized.


In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
import torch

# Ensure GPU is used
def must_use_gpu():
    if not torch.cuda.is_available():
        raise EnvironmentError("No GPU found! Ensure a GPU is available for training.")
    else:
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")

# Check for GPU
must_use_gpu()

# Load Model
device = torch.device("cuda")
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
print("Model loaded.")

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=200,
    save_steps=1000,
    learning_rate=3e-5,
    fp16=True,  # Mixed Precision Training
    report_to="none",
    dataloader_num_workers=4,
    gradient_accumulation_steps=2,
    save_total_limit=2,
)

print("Training arguments configured.")


Using GPU: NVIDIA A100-SXM4-40GB
Model loaded.
Training arguments configured.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_t8")
tokenizer.save_pretrained("./fine_tuned_t8")

print("Model fine-tuned and saved.")


Step,Training Loss,Validation Loss
500,0.256200,0.140543


Model fine-tuned and saved.


In [ ]:
# Evaluate model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


Evaluation Results: {'eval_loss': 0.13030636310577393, 'eval_runtime': 5.5003, 'eval_samples_per_second': 363.437, 'eval_steps_per_second': 22.726, 'epoch': 3.0}
